In [1]:
import sys
import numpy as np
import sklearn.linear_model as skl
import pickle

In [2]:
import argparse
import sys

# Check if the script is being run in an interactive environment
if 'ipykernel' in sys.modules:
    # Manually set the 'sub' variable
    sub = 2  # You can change this value as needed
else:
    # Use argparse as usual
    parser = argparse.ArgumentParser(description='Argument Parser')
    parser.add_argument("-sub", "--sub", help="Subject Number", default=1, type=int)
    args = parser.parse_args()
    sub = args.sub

# Ensure that sub is within the allowed range
assert sub in [1, 2, 5, 7], "The value of 'sub' must be one of [1, 2, 5, 7]"

print(f"The value of 'sub' is: {sub}")


The value of 'sub' is: 2


In [4]:
import os
os.chdir(r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush')
print(os.getcwd())  # Verify the new working directory

C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush


In [5]:
train_path = 'Data/processed_nsddata/subj{:02d}/Auditory/nsd_train_fmriavg_Auditory_sub{}.npy'.format(sub,sub)
train_fmri = np.load(train_path)
test_path = 'Data/processed_nsddata/subj{:02d}/Auditory/nsd_test_fmriavg_Auditory_sub{}.npy'.format(sub,sub)
test_fmri = np.load(test_path)

In [6]:
train_fmri = train_fmri/300
test_fmri = test_fmri/300


norm_mean_train = np.mean(train_fmri, axis=0)
norm_scale_train = np.std(train_fmri, axis=0, ddof=1)
train_fmri = (train_fmri - norm_mean_train) / norm_scale_train
test_fmri = (test_fmri - norm_mean_train) / norm_scale_train

print(np.mean(train_fmri),np.std(train_fmri))
print(np.mean(test_fmri),np.std(test_fmri))

print(np.max(train_fmri),np.min(train_fmri))
print(np.max(test_fmri),np.min(test_fmri))

num_voxels, num_train, num_test = train_fmri.shape[1], len(train_fmri), len(test_fmri)


train_clip = np.load('Data/extracted_features/subj{:02d}/nsd_cliptext_train.npy'.format(sub))
test_clip = np.load('Data/extracted_features/subj{:02d}/nsd_cliptext_test.npy'.format(sub))

## Regression
num_samples,num_embed,num_dim = train_clip.shape

print("Training Regression")
reg_w = np.zeros((num_embed,num_dim,num_voxels)).astype(np.float32)
reg_b = np.zeros((num_embed,num_dim)).astype(np.float32)
pred_clip = np.zeros_like(test_clip)
for i in range(num_embed):
    reg = skl.Ridge(alpha=100000, max_iter=50000, fit_intercept=True)
    reg.fit(train_fmri, train_clip[:,i])
    reg_w[i] = reg.coef_
    reg_b[i] = reg.intercept_

    pred_test_latent = reg.predict(test_fmri)
    std_norm_test_latent = (pred_test_latent - np.mean(pred_test_latent,axis=0)) / np.std(pred_test_latent,axis=0)
    pred_clip[:,i] = std_norm_test_latent * np.std(train_clip[:,i],axis=0) + np.mean(train_clip[:,i],axis=0)
    print(i,reg.score(test_fmri,test_clip[:,i]))


-2.6905091170276335e-16 0.9999435586284333
-0.0230087925174602 0.9934162733048788
14.987235205038967 -14.95580829898399
13.847471245936827 -12.376000889458274
Training Regression
0 0.06993177417731848
1 0.01822931908843763
2 0.052135685510946994
3 0.05377105857689111
4 0.05859353957229068
5 0.05774858681304893
6 0.05834531847660949
7 0.06154981234336778
8 0.060961959992760356
9 0.06191740853973245
10 0.07223775591076305
11 0.08637742416649245
12 0.10271776258650255
13 0.11716067006961788
14 0.12543152143011935
15 0.1292312860813554
16 0.13259890256395382
17 0.13549090768958127
18 0.1369092006789906
19 0.13775770238055574
20 0.13791385655982688
21 0.1381249905046467
22 0.1380687477426142
23 0.13798756345959437
24 0.1380546211061839
25 0.1381718080976424
26 0.1380960306916483
27 0.13782881438007694
28 0.13761303798435784
29 0.1374266295169456
30 0.13728404410491013
31 0.13715125556015867
32 0.13697496055474373
33 0.13687595280358744
34 0.13680055635592525
35 0.13674839079336176
36 0.1367

In [7]:
import os
import numpy as np
import pickle

base_path = r'C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results'

# === Save predicted features ===
pred_path = os.path.join(base_path, 'predicted_features', f'subj{sub:02d}')
os.makedirs(pred_path, exist_ok=True)

pred_file = os.path.join(pred_path, 'nsd_cliptext_predtest_nsdgeneral.npy')
np.save(pred_file, pred_clip)

# === Save regression weights ===
reg_path = os.path.join(base_path, 'regression_weights', f'subj{sub:02d}')
os.makedirs(reg_path, exist_ok=True)

datadict = {
    'weight': reg_w,
    'bias': reg_b,
}

reg_file = os.path.join(reg_path, 'cliptext_regression_weights.pkl')
with open(reg_file, "wb") as f:
    pickle.dump(datadict, f)